## Configure PUDL
The `.pudl.yml` configuration file tells PUDL where to look for data. Uncomment the next cell and run it if you're on our 2i2c JupyterHub.

In [1]:
#!cp ~/shared/shared-pudl.yml ~/.pudl.yml

In [2]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

In [3]:
# setup for python logging
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

## Set your EIA API Key
Some of the routines in this notebook pull data from the EIA API to fill in missing fuel cost values. For them to work, you'll need to provide an API KEY. You can [obtain one from EIA here](https://www.eia.gov/opendata/register.php), uncomment the code in the next cell, and use it to set the `API_KEY_EIA` environment variable to be your key. (If you're running this notebook on your own computer and want to set the environment variable outside of the notebook [see this blog post](https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html))

To set your API key in this notebook, uncomment the first line in the next cell and add your key in the quotes

In [4]:
# os.environ["API_KEY_EIA"] = "put your API key here"
assert os.environ.get("API_KEY_EIA") is not None

# Using the PUDL output layer
The PUDL database tables are a clean, [normalized](https://en.wikipedia.org/wiki/Database_normalization) version of US electricity data. Normalized tables are great for databases and storage, but for interactive use, we often want a version of the data that includes plant and utility names and other associated info all in a single dataframe. These are "denormalized" tables. In addition to the referenced names and attributes like latitude and longitude or state, the denormalized tables often contain frequently calculated derived values (like calcuating `total_fuel_cost` from `total_heat_content_mmbtu` and `fuel_cost_per_mmbtu`). The Catalyst team developed a useful tool to access denormalized tables that we call the PUDL output object.

## What does the output layer provide?

Right now the output layer provides access to three different kinds of things:
 * denormalized tables
 * analytical outputs
 * partially integrated PUDL datasets that aren't in the database yet

## Why is the output layer useful?
Some benefits of using the output layer:
 * **Standardized denormalization:** You don't have to manually join the same tables together to get access to common attributes.
 * **Table caching:** many analyses rely on using the same table multiple times. The PUDL output object caches the tables in memory as pandas dataframes so you don't have to read tables from the database over and over again.
 * **Time series aggregation:** Some tables are annual, some monthly, some hourly. When you create a PUDL output object you can tell it to aggregate the data to either monthly or annual resolution for analysis.
 * **Standardized the filling-in of missing data:** There's a ton of missing or incomplete data. If requested, the output objects will use rolling averages and  data from the EIA API try to fill some of that missing data in.

# Instantiating Output Objects
* Grab the `pudl_settings`
* Create a connection to the PUDL Database
* Instantiate a `PudlTabl` object with that connection

In [5]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'datapkg_dir': '/home/zane/code/catalyst/pudl-work/datapkg',
 'notebook_dir': '/home/zane/code/catalyst/pudl-work/notebook',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite'}

In [6]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_engine

Engine(sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite)

In [7]:
# this configuration will return tables without aggregating by a time frequency... we'll explore that more below.
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## List the output object methods
* There are dozens of different data access methods within the `PudlTabl` object. If you want to see all of them with their docstrings, you can un-comment and run `help(pudl_out)` in the next cell.
* If you type `pudl_out.` and press `Shift` and `Tab` at the same time, you'll see a list of available methods as well.

In [8]:
#help(pudl_out)

This cell will print out a simple list of all the available public methods inside the `pudl_out` object

In [9]:
# this is the master list of all of the methods in the pudl_out object
# they all return a table cooresponding to their name
methods_pudl_out = [
    method_name for method_name in dir(pudl_out)
    if callable(getattr(pudl_out, method_name))    # if it is a method
    and '__' not in method_name                    # remove the internal methods
]
methods_pudl_out

['adjacency_ba_ferc714',
 'advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'bf_eia923',
 'bga',
 'bga_eia860',
 'capacity_factor',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'description_pa_ferc714',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'etl_eia861',
 'etl_ferc714',
 'fbp_ferc1',
 'frc_eia923',
 'fuel_cost',
 'fuel_ferc1',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gen_plants_ba_ferc714',
 'gens_eia860',
 'gf_eia923',
 'green_pricing_eia861',
 'hr_by_gen',
 'hr_by_unit',
 'id_certification_ferc714',
 'interchange_ba_ferc714',
 'lambda_description_ferc714',
 'lambda_hourly_ba_ferc714',
 'mcoe',
 'mergers_eia861',
 'net_energy_load_ba_ferc714',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_dat

## Basic Functionality

### Read a denormalized table
* Each of output object methods will return a Pandas Dataframe.
* Most of them correspond to a single database table, and will select all the data in that table, and automatically join it with some other useful information.
* Many of the access methods use an abbreviated name for the database table. E.g. the following reads all the data out of the `generators_eia860` table.

In [10]:
%%time
gens_eia860 = pudl_out.gens_eia860().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403888 entries, 403887 to 0
Columns: 110 entries, report_date to zip_code
dtypes: Int64(4), datetime64[ns](1), float64(21), int64(1), object(83)
memory usage: 343.6+ MB
CPU times: user 51.4 s, sys: 5.12 s, total: 56.5 s
Wall time: 1min 5s


### Automatic dataframe caching
The `generators_eia860` table is quite long, and the above cell probably took several seconds to read 270,000 records each with 100 columns, creating an 800MB Dataframe. If you run the same output routine again, it will complete almost instantly because that dataframe is already stored inside `pudl_out`. This is memory intensive, but can save time in calculations that need to use the same tables several times.

In [11]:
%%time
gens_again_eia860 = pudl_out.gens_eia860()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


## Exploring `pudl_out` Arguments
Below, we'll explore the main arguments that are used to customize the PUDL output object. You can mix and match these options.

By default, the output object will read data from all available years, do no time aggregation, and not attempt to fill in missing values.

In [12]:
# here are the default arguments for the pudl_out object
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq=None,               # Desired time grouping to aggregate PUDL tables to.
    start_date=None,         # Beginning date for data to pull from the PUDL DB.
    end_date=None,           # End date for data to pull from the PUDL DB.
    fill_fuel_cost=False,    # Whether to fill in missing fuel costs with EIA monthly state-level averages.
    roll_fuel_cost=False,    # Whether to fill in monthly missing fuel costs with a 12-month rolling average.
)

### Time series aggregation
The PUDL output object can aggregate data on a monthly or annual basis, if you set the `freq` argument to `AS` (annual starting at the beginning of the calendar year) or `MS` (monthly starting at the beginning of the month) or [other equivalent frequency abbreviations](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases).

**NOTE:** Not all columns can be aggregated, so you may lose access to some kinds of information in aggregated outputs. If you need to retain information that gets lost in the default aggregation / groupby process, you may need to pull the unaggregated data and do your own aggregation.

In [13]:
pudl_out_as = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='AS',               # Aggregate tables annually
)

In [14]:
pudl_out_as.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,1,221908.0
1,2009-01-01,3,32,Barry,195,18,Alabama Power Co,2,394031.0
2,2009-01-01,3,32,Barry,195,18,Alabama Power Co,3,1286393.0
3,2009-01-01,3,32,Barry,195,18,Alabama Power Co,4,1626547.0
4,2009-01-01,3,32,Barry,195,18,Alabama Power Co,5,4513101.0


In [15]:
pudl_out_ms = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
)

In [16]:
pudl_out_ms.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,1,39699.0
1,2009-02-01,3,32,Barry,195,18,Alabama Power Co,1,5594.0
2,2009-03-01,3,32,Barry,195,18,Alabama Power Co,1,13015.0
3,2009-04-01,3,32,Barry,195,18,Alabama Power Co,1,15858.0
4,2009-05-01,3,32,Barry,195,18,Alabama Power Co,1,68232.0


### Filling in Missing Fuel Costs
 * The original EIA data is often incomplete.
 * Many utilities withold information about their fuel costs.
 * We have a couple of ways of estimating missing values, if you need complete data.

The ouput object created in the next cell will attempt to use all of these methods to fill in missing data.
To fill in missing fuel costs, we can pull monthly state-level average fuel costs from EIA, and we can use rolling averages to fill in short gaps in the data.
* Set `fill_fuel_cost=True` when creating an output object to pull average monthly fuel costs from the EIA API.
* Set `roll_fuel_cost=True` when creating an output object to use a 12-month rolling average based on available data to fill in gaps.
* These options can be used together to fill in as many gaps as possible.
* **NOTE:** You need to have set the `API_KEY_EIA` environment variable to a valid EIA API key for this to work. See instructions at the top of this notebook.

In [17]:
pudl_out_fill = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
    fill_fuel_cost=True,     # Fill in missing fuel cost records with state-level averages from EIA's API
    roll_fuel_cost=True,     # Fill in missing fuel cost records with a 12-month rolling average.
)

In [18]:
%%time
pudl_out_fill.frc_eia923().head()

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 3min 7s, sys: 2.14 s, total: 3min 10s
Wall time: 3min 34s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,fuel_qty_units,fuel_type_code_pudl,heat_content_mmbtu_per_unit,mercury_content_ppm,moisture_content_pct,sulfur_content_pct,total_fuel_cost,total_heat_content_mmbtu
0,2009-01-01,3,32,Barry,195,18,Alabama Power Co,10.013475,NaN,False,4.526190,381438.0,coal,23.334763,NaN,NaN,0.938976,4.028655e+07,8900765.245
1,2009-02-01,3,32,Barry,195,18,Alabama Power Co,9.026785,NaN,False,4.096987,410147.0,coal,23.056621,NaN,NaN,0.822421,3.874359e+07,9456604.085
2,2009-03-01,3,32,Barry,195,18,Alabama Power Co,6.449671,NaN,False,3.709062,376787.0,coal,22.876760,NaN,NaN,0.487466,3.197087e+07,8619665.590
3,2009-04-01,3,32,Barry,195,18,Alabama Power Co,7.520152,NaN,False,3.897879,105322.0,coal,23.160458,NaN,NaN,0.523833,9.508120e+06,2439305.763
4,2009-05-01,3,32,Barry,195,18,Alabama Power Co,6.669016,NaN,False,3.672290,367333.0,coal,22.899997,NaN,NaN,0.621304,3.089103e+07,8411924.736


Looking at the filled vs. unfilled monthly data in the Fuel Receipts and Costs data from EIA 923, we can see that there are about 190k possible monthly records. Unfilled, we have fuel costs for about 107k of them. With the state level monthly fuel costs and rolling averages, we can get that up to about 116k records. An improvement, but it's not great. Unfortunately this data simply isn't reported publicly.

In [19]:
pudl_out_ms.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190115 entries, 0 to 190114
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         190115 non-null  Int64         
 1   report_date          190115 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  106695 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 6.0 MB


In [20]:
pudl_out_fill.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190115 entries, 0 to 190114
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         190115 non-null  Int64         
 1   report_date          190115 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  115786 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 6.0 MB


## Free Memory
Because this JupyterHub has limited memory, we need to delete the cached dataframes when we're done with them.

In [21]:
del pudl_out
del pudl_out_ms
del pudl_out_as
del pudl_out_fill

# Denormalized Output Tables
* Below, we'll extract and show a sample of many of the available denormalized PUDL output tables.
* If you'd like to see more than 5 sample rows, feel free to change `n_samples` below.
* Rather than assigning the results of these functions to a local variable in the notebook, we're showing samples from the cached dataframes to conserve memory, as this JupyterHub has limited RAM available at the moment.

In [22]:
n_samples = 5
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## EIA Forms 860 & 923

In [23]:
# here are all of the EIA tables
tables_eia = [
    t for t in methods_pudl_out 
    if '_eia' in t 
    and '_eia861' not in t       # avoid the EIA 861 tables for now bc it is preliminary
]
tables_eia

['bf_eia923',
 'bga_eia860',
 'frc_eia923',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gens_eia860',
 'gf_eia923',
 'own_eia860',
 'plants_eia860',
 'pu_eia860',
 'utils_eia860']

### EIA Plant Utility Associations
This is mostly a helper function, used for adding plant and utility names and IDs into other output tables.

In [24]:
%%time
pudl_out.pu_eia860().sample(n_samples)

CPU times: user 7.93 s, sys: 16 ms, total: 7.95 s
Wall time: 8.01 s


,report_date,plant_id_eia,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl
19404,2006-01-01,2051,Delta (MS),153,12685,Entergy Mississippi LLC,109
105068,2013-01-01,57419,RPI Fuel Cell LLC,5726,56748,RP1 Fuel Cell LLC,3055
120272,2013-01-01,59227,Optimum Wind 3 LLC,10655,59025,Optimum Wind 3 LLC,5143
31477,2020-01-01,3611,O W Sommers,2385,16604,San Antonio Public Service Bd,1178
83404,2019-01-01,55557,Wilmington Hydrogen Plant,4592,991,Air Products & Chemicals Inc,427


### EIA 860 Boiler Generator Associations
* **NOTE:** We have filled in many more boiler-generator associations based on additional information. The `bga_source` column indicates where the association came from.

In [25]:
%%time
pudl_out.bga_eia860().sample(n_samples)

CPU times: user 707 ms, sys: 8.34 ms, total: 716 ms
Wall time: 731 ms


,plant_id_eia,report_date,generator_id,boiler_id,unit_id_eia,unit_id_pudl,bga_source
37586,10521,2019-01-01,GEN2,GEN2,CC1,1,string_assn
39186,10606,2018-01-01,GEN5,9A,None,1,eia860_org
57122,50561,2018-01-01,STG2,0002,CC1,1,eia860_org
91098,55501,2014-01-01,STG2,CTG3,STG2,2,eia860_org
70985,54349,2018-01-01,STM,GTA,STG1,1,eia860_org


### EIA 860 Plants

In [26]:
%%time
pudl_out.plants_eia860().sample(n_samples)

CPU times: user 3.87 s, sys: 32 ms, total: 3.9 s
Wall time: 3.92 s


,plant_id_eia,plant_name_eia,balancing_authority_code_eia,balancing_authority_name_eia,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,...,pipeline_notes,regulatory_status_code,transmission_distribution_owner_id,transmission_distribution_owner_name,transmission_distribution_owner_state,utility_id_eia,water_source,plant_id_pudl,utility_name_eia,utility_id_pudl
88733,56002,Minwind,MISO,Midcontinent Independent Transmission System O...,Luverne,Rock,False,False,False,24.5,...,None,NR,56088,Alliant Energy,IA,<NA>,None,4749,NaN,<NA>
5106,437,Edward C Hyatt,CISO,California Independent System Operator,Oroville,Butte,False,False,False,230.0,...,None,RE,14328,None,CA,3255,Water From Penstocks Oroville,1464,California Dept. of Water Resources,743
74546,54713,Alliant SBD 8603 Donnelley,MISO,Midcontinent Independent Transmission System O...,Nevada,Story,False,False,False,NaN,...,None,NR,None,None,None,361,None,10108,Industrial Energy Applications Inc,2085
138660,63220,USS Water City Solar LLC CSG,MISO,Midcontinent Independent Transmission System O...,Waterville,Le Sueur,False,False,True,34.5,...,None,NR,13781,Northern States Power Co - Minnesota,MN,62978,None,14914,NaN,<NA>
118906,58925,Sangamon Wind One LLC,MISO,Midcontinent Independent Transmission System O...,TBD,Sangamon,False,False,False,69.0,...,None,NR,56697,Ameren Illinois Company,IL,60569,None,10718,NaN,<NA>


### EIA 860 Generators

In [27]:
%%time
pudl_out.gens_eia860().sample(n_samples)

CPU times: user 40.1 s, sys: 3.92 s, total: 44 s
Wall time: 44.1 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,topping_bottoming_code,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
309210,2008-01-01,50621,7921,Central Power Plant,18000,4071,State of Rhode Island,GEN6,True,ISNE,...,None,None,NaN,None,None,None,None,3.8,NaN,02920
182110,2014-01-01,3423,2343,Cordell Hull,19462,3622,USCE-Nashville District,2,False,TVA,...,X,0.0,NaN,None,None,False,None,38.0,NaN,37030
374547,2005-01-01,7145,457,Pleasant Hill,12341,185,MidAmerican Energy Co,1,False,MISO,...,X,None,NaN,None,None,None,None,47.0,NaN,50317
322423,2008-01-01,317,1040,Big Creek 1,17609,295,Southern California Edison Co,3,False,CISO,...,X,None,NaN,None,None,None,None,17.2,NaN,93605
197454,2013-01-01,54908,4298,Enderlin,772,522,Archer Daniels Midland Co,GEN2,True,MISO,...,T,None,NaN,None,None,False,None,4.7,NaN,58027


### EIA 860 Generator-level Ownership

In [28]:
%%time
pudl_out.own_eia860().sample(n_samples)

CPU times: user 7.92 s, sys: 140 ms, total: 8.06 s
Wall time: 8.1 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,owner_utility_id_eia,owner_name,fraction_owned,owner_city,owner_state,owner_street_address,owner_zip_code
7438,2006-01-01,1713,886,Ludington,4254,81,Consumers Energy Co,3,5109,Detroit Edison Co,0.49,None,None,None,None
59912,2018-01-01,59972,10346,Bearkat,61006,6125,Bearkat TE Partnership LLC,BRKAT,61007,Bearkat I TE Partnership LLC,1.00,Austin,TX,"701 Brazos Street, Suite 1400",78701
64223,2019-01-01,58866,6865,Garrell Solar Farm,58743,1844,Garrell Solar Farm LLC,1,58935,"Strata Master Lessee 1, LLC",1.00,Chapel Hill,NC,50101 Governors Drive,27517
44902,2015-01-01,55239,4468,AES Red Oak LLC,22131,2970,Red Oak Power LLC,0003,59555,"Red Oak Power, LLC",1.00,Charlotte,NC,9405 Arrowpoint Blvd,28273
28614,2011-01-01,3494,2361,Permian Basin,55983,2328,Luminant Generation Company LLC,5,57096,Luminant Gen Co LLC Fin Holding,1.00,None,None,None,None


### EIA 923 Generation and Fuel Consumption

In [29]:
%%time
pudl_out.gf_eia923().sample(n_samples)

CPU times: user 23.9 s, sys: 2.36 s, total: 26.3 s
Wall time: 26.7 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,fuel_consumed_for_electricity_mmbtu,fuel_consumed_for_electricity_units,fuel_consumed_mmbtu,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type,fuel_type_code_aer,fuel_type_code_pudl,net_generation_mwh,nuclear_unit_id,prime_mover_code
617593,2014-02-01,93,1331,Riley#03a,17271,893,Sitka City & Borough of,32101.0,0.0,32101.0,0.0,0.000,WAT,HYC,hydro,3375.521,NaN,HY
474653,2012-06-01,57797,6012,Golden Springs Building D,57104,1901,Golden Springs Development Company LLC,1096.0,0.0,1096.0,0.0,0.000,SUN,SUN,solar,115.131,NaN,PV
696010,2014-11-01,50656,3751,Huntington Resource Recovery Facility,9097,2046,Huntington Resource Recovery,730.0,292.0,730.0,292.0,2.500,PG,WOO,gas,40.199,NaN,ST
476032,2012-05-01,57916,6103,Spreckels Sugar Company,57299,3262,Spreckels Sugar Company,16795.0,16612.0,74323.0,73514.0,1.011,NG,NG,gas,1432.387,NaN,ST
830295,2015-04-01,50215,3544,York County Resource Recovery,21128,3830,York County Solid W & R Auth,169862.0,23684.0,169862.0,23684.0,7.172,MSB,MLG,waste,9300.673,NaN,ST


### EIA 923 Fuel Receipts and Costs

In [30]:
%%time
pudl_out.frc_eia923().sample(n_samples)

CPU times: user 23.3 s, sys: 595 ms, total: 23.9 s
Wall time: 24 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,contract_expiration_date,...,moisture_content_pct,natural_gas_delivery_contract_type_code,natural_gas_transport_code,primary_transportation_mode_code,secondary_transportation_mode_code,state,sulfur_content_pct,supplier_name,total_fuel_cost,total_heat_content_mmbtu
111250,2011-07-01,535,1519,McClellan (CA),16534,3069,Sacramento Municipal Util Dist,0.00,NaN,NaT,...,NaN,<NA>,firm,PL,<NA>,<NA>,0.00,shell energy north america,954.100000,203.000
107162,2010-05-01,7872,2942,Robert P Mone Plant,25422,2553,National Power Coop Inc,0.00,NaN,NaT,...,NaN,<NA>,interruptible,PL,<NA>,<NA>,0.00,anr,175069.679616,39716.352
376775,2017-03-01,55294,4497,Westbrook Energy Center,2891,3736,Calpine Operating Services Cpompany Inc,0.00,NaN,NaT,...,NaN,firm,firm,PL,<NA>,<NA>,0.00,calpine energy,NaN,1080780.580
154407,2011-10-01,50974,3882,Scrubgrass Generating,14932,3613,US Operating Services Company,15.60,NaN,NaT,...,NaN,<NA>,<NA>,TR,<NA>,PA,2.24,allegheny mineral corp,NaN,2419.200
223974,2013-09-01,6166,530,Rockport,9324,144,Indiana Michigan Power Co,12.16,NaN,NaT,...,NaN,<NA>,<NA>,RV,<NA>,KY,0.96,"rwe trading americas,inc.",521385.187904,166099.136


### EIA 923 Boiler Fuel Consumption

In [31]:
%%time
pudl_out.bf_eia923().sample(n_samples)

CPU times: user 17.3 s, sys: 1.05 s, total: 18.3 s
Wall time: 18.4 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,boiler_id,ash_content_pct,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code,fuel_type_code_pudl,sulfur_content_pct,total_heat_content_mmbtu
16421,2009-06-01,1743,599,St Clair,5109,89,The DTE Electric Company,3,0.0,0.0,0.000,RFO,oil,0.0,0.000
990258,2018-07-01,54099,3997,Ticonderoga Mill,9350,2138,IPC-Ticonderoga,PB1,0.0,0.0,0.000,BLQ,waste,0.0,0.000
883221,2017-10-01,54638,4172,Johnsonburg Mill,56137,1501,Domtar LLC,PC81,0.0,0.0,0.000,BLQ,waste,0.0,0.000
720410,2016-03-01,7266,8125,Woodland,12745,2494,Modesto Irrigation District,3,0.0,51356.0,1.034,NG,gas,0.0,53102.104
468074,2014-03-01,3631,2390,Sam Rayburn,17583,3246,South Texas Electric Coop Inc,3,0.0,0.0,0.000,NG,gas,0.0,0.000


### EIA 923 Net Generation by Generator

In [32]:
%%time
pudl_out.gen_eia923().sample(n_samples)

CPU times: user 11.5 s, sys: 280 ms, total: 11.7 s
Wall time: 11.8 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh
377926,2017-11-01,56708,5225,CFB Power Plant,6541,1748,Formosa Plastics Corp,G2101,0.0
272108,2015-09-01,55086,4379,Gregory Power Plant,7667,1939,Gregory Power Partners LLC,GT1B,101491.0
471564,2019-01-01,56356,4960,Clearwater Power Plant,16088,1168,Riverside City of,CT1,NaN
187632,2014-01-01,1010,613,Wabash River,15470,92,PSI Energy Inc,5,24851.0
161435,2013-12-01,10075,578,Taconite Harbor Energy Center,12647,23,"ALLETE, Inc.",GEN2,39800.0


## FERC Form 1
* Only a small subset of the 100+ tables that exist in the original FERC Form 1 have been cleaned and included in the PUDL DB.
* For tables not included here, you'll need to access the cloned multi-year FERC 1 DB that we produce. See the first tutorial notebook for more information.

In [33]:
# All of the FERC Form 1 tables end with _ferc1
tables_ferc1 = [
    t for t in methods_pudl_out 
    if '_ferc1' in t 
]
tables_ferc1

['fbp_ferc1',
 'fuel_ferc1',
 'plant_in_service_ferc1',
 'plants_hydro_ferc1',
 'plants_pumped_storage_ferc1',
 'plants_small_ferc1',
 'plants_steam_ferc1',
 'pu_ferc1',
 'purchased_power_ferc1']

### FERC 1 Large Steam Plants
The large steam plants report detailed operating expenses in this table, as well as operational characteristics.

In [34]:
%%time
pudl_out.plants_steam_ferc1().sample(n_samples)

CPU times: user 796 ms, sys: 19.8 ms, total: 816 ms
Wall time: 833 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw
10665,1996,120,224,Northern States Power Company (Minnesota),646,908,wilmarth,NaN,25.0,0.473151,...,715946.0,NaN,82742.0,NaN,11.0,NaN,8230.0,steam,f1_steam_1996_12_120_0_3,22.0
16282,2000,135,249,PECO Energy Company,3008,1232,croyden,NaN,NaN,0.006731,...,NaN,NaN,28889.0,NaN,425.0,NaN,296.0,internal_combustion,f1_steam_2000_12_135_3_2,370.0
5166,1999,126,234,Ohio Edison Company,7887,443,edgewater,NaN,21.0,0.184612,...,818020.0,NaN,396.0,NaN,106.0,100.0,2787.0,steam,f1_steam_1999_12_126_0_2,100.0
13808,2002,17,97,"Duke Energy Progress, Inc.",76,86,brunswick,NaN,764.0,0.878620,...,11639967.0,NaN,6841761.0,NaN,3459.0,NaN,8735.0,nuclear,f1_steam_2002_12_17_3_1,1378.0
27228,2019,194,364,Wisconsin Power and Light Company,512,5897,riverside unit 2,26994.0,43.0,0.677099,...,54442.0,NaN,485600.0,NaN,186.0,187.0,7704.0,combustion_turbine,f1_steam_2019_12_194_1_2,NaN


### FERC 1 Fuel
Fuel consumption by the large steam plants, broken down by plant and fuel type.

In [35]:
%%time
pudl_out.fuel_ferc1().sample(n_samples)

CPU times: user 357 ms, sys: 20 ms, total: 377 ms
Wall time: 378 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,fuel_consumed_mmbtu,fuel_consumed_total_cost,fuel_cost_per_mmbtu,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_mmbtu_per_unit,fuel_qty_burned,fuel_type_code_pudl,fuel_unit,record_id
6819,1996,173,331,TXU Electric Company,2361,permian basin,2.597563e+07,6.964365e+07,2.680,2.730,2.730,1.018233,25510494.0,gas,mcf,f1_fuel_1996_12_173_1_13
2598,2000,70,140,Idaho Power Company,605,valmy,1.845686e+07,2.946677e+07,1.630,33.230,34.342,20.814000,886752.0,coal,ton,f1_fuel_2000_12_70_0_7
2037,2003,51,318,The Empire District Electric Company,295,iatan,5.802014e+03,3.611201e+04,6.224,36.004,36.004,5.784660,1003.0,oil,bbl,f1_fuel_2003_12_51_0_14
20956,2002,79,159,Kansas City Power & Light Company,652,wolf creek,4.376036e+01,1.767919e+07,0.404,0.404,0.404,0.000001,43760363.0,nuclear,mmbtu,f1_fuel_2002_12_79_2_1
28727,2012,227,368,"Wolverine Power Supply Cooperative, Inc.",568,sumpter,1.429109e+06,7.700998e+06,3.577,5.518,3.668,1.024000,1395614.0,gas,mcf,f1_fuel_2012_12_227_1_4


### FERC 1 Fuel by Plant
Wide-form aggregated fuel totals by plant and year, identifying the relative cost and heat content proportions of different fuels, as well as the primary fuel for the plant.

In [36]:
%%time
pudl_out.fbp_ferc1().sample(n_samples)

CPU times: user 677 ms, sys: 4.09 ms, total: 681 ms
Wall time: 695 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,coal_fraction_cost,coal_fraction_mmbtu,fuel_cost,fuel_mmbtu,...,nuclear_fraction_cost,nuclear_fraction_mmbtu,oil_fraction_cost,oil_fraction_mmbtu,primary_fuel_by_cost,primary_fuel_by_mmbtu,unknown_fraction_cost,unknown_fraction_mmbtu,waste_fraction_cost,waste_fraction_mmbtu
12824,2006,150,277,"Puget Sound Energy, Inc.",205,fredonia 3 & 4,0.000000,0.000000,2.946926e+06,3.022622e+05,...,0.0,0.0,0.089093,0.132032,gas,gas,0.0,0.0,0.0,0.0
14302,2002,166,302,Southwestern Public Service Company,251,harrington station,0.000000,0.000000,8.034358e+05,2.415339e+05,...,0.0,0.0,0.000000,0.000000,gas,gas,0.0,0.0,0.0,0.0
9358,2010,120,224,Northern States Power Company (Minnesota),6,a s king,0.998382,0.999391,6.918466e+07,3.442510e+07,...,0.0,0.0,0.000130,0.000020,coal,coal,0.0,0.0,0.0,0.0
7988,2004,89,171,Madison Gas and Electric Company,464,portable generators,0.000000,0.000000,8.569510e+04,6.887385e+03,...,0.0,0.0,1.000000,1.000000,oil,oil,0.0,0.0,0.0,0.0
8808,1998,101,191,MONONGAHELA POWER COMPANY,1292,willow island,1.000000,1.000000,1.096613e+07,9.238836e+06,...,0.0,0.0,0.000000,0.000000,coal,coal,0.0,0.0,0.0,0.0


### FERC 1 Plant in Service
An accounting of how much electric plant infrastructure exists in each of the many FERC accounts. This is a very wide form table.

In [37]:
%%time
pudl_out.plant_in_service_ferc1().sample(n_samples)

CPU times: user 1.15 s, sys: 19.8 ms, total: 1.17 s
Wall time: 1.2 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,record_id,amount_type,distribution_acct360_land,distribution_acct361_structures,distribution_acct362_station_equip,distribution_acct363_storage_battery_equip,...,transmission_acct352_structures,transmission_acct353_station_equip,transmission_acct354_towers,transmission_acct355_poles,transmission_acct356_overhead_conductors,transmission_acct357_underground_conduit,transmission_acct358_underground_conductors,transmission_acct359_1_asset_retirement,transmission_acct359_roads_trails,transmission_total
23229,2004,270,268,PSEG Fossil LLC,f1_plant_in_srvce_2004_12_270_0,starting_balance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23563,2016,276,359,"Westar Generating, Inc.",f1_plant_in_srvce_2016_12_276_0,ending_balance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12109,1995,121,225,Northern States Power Company (Wisconsin),f1_plant_in_srvce_1995_12_121_0,retirements,2162.0,NaN,395908.0,NaN,...,NaN,311626.0,14370.0,241191.0,230071.0,NaN,NaN,NaN,NaN,799234.0
9290,2006,91,173,Maine Public Service Company,f1_plant_in_srvce_2006_12_91_0,retirements,NaN,NaN,7484.0,NaN,...,NaN,39932.0,NaN,124115.0,21290.0,NaN,NaN,NaN,NaN,185337.0
12077,2015,121,225,Northern States Power Company (Wisconsin),f1_plant_in_srvce_2015_12_121_0,starting_balance,2125123.0,9874515.0,147511956.0,NaN,...,19418483.0,245372918.0,2970127.0,271459411.0,137130678.0,6313729.0,9044585.0,26282.0,26067.0,718130266.0


### FERC 1 Purchased Power
A summary of electricity market transactions between utilities. Sadly the sellers are identified only by their names, and not their FERC Utility (Respondent) ID.

In [38]:
%%time
pudl_out.purchased_power_ferc1().sample(n_samples)

CPU times: user 1.74 s, sys: 27.5 ms, total: 1.77 s
Wall time: 1.8 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,seller_name,record_id,billing_demand_mw,coincident_peak_demand_mw,delivered_mwh,demand_charges,energy_charges,non_coincident_peak_demand_mw,other_charges,purchase_type,purchased_mwh,received_mwh,tariff,total_settlement
5206,2001,7,31,Arizona Public Service Company,Grant County,f1_purchased_pwr_2001_12_7_3_11,NaN,NaN,0.0,0.0,0.0,NaN,-2054928.0,electricity_exchange,0.0,0.0,None,-2054928.0
35413,2012,70,140,Idaho Power Company,Glenns Ferry Cogen Partners/Magic,f1_purchased_pwr_2012_12_70_2_13,NaN,NaN,0.0,0.0,-7850.0,NaN,0.0,long_unit,-166.0,0.0,-,-7850.0
162898,2008,45,90,"Duke Energy Carolinas, LLC",Mark A Powers,f1_purchased_pwr_2008_12_45_2_13,NaN,NaN,0.0,0.0,45.0,NaN,0.0,long_unit,2.0,0.0,Note (1),45.0
1805,2002,115,213,New York State Electric & Gas Corporation,Finger Lakes Energy,f1_purchased_pwr_2002_12_115_2_1,NaN,NaN,0.0,0.0,48900.0,NaN,0.0,other_service,545.0,0.0,None,48900.0
101562,2012,181,340,Upper Peninsula Power Company,Parallel Generation,f1_purchased_pwr_2012_12_181_0_4,NaN,NaN,0.0,0.0,3461.0,NaN,0.0,other_service,20.0,0.0,None,3461.0


## Free Memory
Again, because we're on a JupyterHub with limited RAM per user, we need to delete the cached dataframes we've just created.

In [39]:
del pudl_out

# Analysis Outputs
* The PUDL Database is mainly meant to standardize the structure of data that's been reported in different ways over different years, so that it can all be used together.
* We typically don't include calculated values or big modifications to the original data.
* We're compiling a growing library of stock analyses in the `pudl.analysis` subpackage, which operate on data stored in the database.
* Some of these analytical outputs are build into the output object so that they can take advantage of the dataframe caching, and for convenient access.

## The Marginal Cost of Electricity (MCOE)
* One of our first analysis modules calculates fuel costs, heat rates, and capacity factors on a generator by generator basis.
* The long term goal is for it to provide a comprehensive marginal cost of electricity production (MCOE).
* The integration of operating costs from FERC Form 1 is still a work in progress, and hasn't been added in here yet.

### MCOE Requires Aggregation
* Fuel costs and other data need to be aggregated by month or year to calculate MCOE.
* This means we need an output object that aggregates by month or year.
* Because a single `NA` value can wipe out a whole aggregated category, you'll get more information with a Monthly aggregation.

In [40]:
pudl_out_monthly = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq="MS",
    fill_fuel_cost=True,
    roll_fuel_cost=True,
)

### Heat Rate by Generation Unit (MMBTU/MWh)
* A "Generation Unit" (identifyed by `unit_id_pudl` here) is a group of "boilers" (where fuel is consumed) and "generators" (where electricity is made) which are connected to each other.
* Because the fuel inputs and electricity outputs are comingled, this is the most granular level at which a direct heat rate calculation can be done.

In [41]:
%%time
pudl_out_monthly.hr_by_unit().sample(n_samples)

CPU times: user 6min 44s, sys: 2.19 s, total: 6min 46s
Wall time: 7min 2s


,report_date,plant_id_eia,unit_id_pudl,net_generation_mwh,total_heat_content_mmbtu,heat_rate_mmbtu_mwh
23437,2009-10-01,50806,1,11167.329,931083.560,83.375672
125842,2013-06-01,2070,1,19670.000,21770.475,1.106786
243363,2017-08-01,55139,1,322086.000,2691955.470,8.357878
67215,2011-06-01,1769,3,33578.000,NaN,NaN
259984,2018-04-01,10837,1,0.000,0.000,NaN


### Heat Rate by Generator (mmBTU/MWh)
* However, we do need per-generator heat rates to estimate per-generator fuel costs.

In [42]:
%%time
pudl_out_monthly.hr_by_gen().sample(n_samples)

CPU times: user 1min 1s, sys: 7.41 s, total: 1min 8s
Wall time: 1min 13s


,report_date,plant_id_eia,heat_rate_mmbtu_mwh,generator_id,fuel_type_code_pudl,fuel_type_count
8575,2009-08-01,1831,13.444815,3,coal,1
355048,2017-05-01,55405,7.380235,ST2,gas,1
416054,2019-03-01,2411,6.821411,701,gas,2
359045,2017-06-01,56407,6.990215,2B,gas,1
185422,2014-11-01,2240,11.623991,8,coal,2


### Per-generator Fuel Costs
* Calculate per-generator fuel costs based on heat rates and fuel deliveries
* Because we told the `pudl_out` object to try and fill in missing values, this will request monthly average fuel cost data by date from the EIA API. It might take a minute.
* This also means you'll need to have set your EIA API Key at the top of the notebook.

In [43]:
%%time
pudl_out_monthly.fuel_cost().sample(n_samples)

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 3min 22s, sys: 3.6 s, total: 3min 26s
Wall time: 3min 41s


,plant_id_eia,report_date,generator_id,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl,fuel_type_count,fuel_type_code_pudl,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,heat_rate_mmbtu_mwh,fuel_cost_per_mwh
162701,50392,2013-06-01,TG1,Eielson AFB Central Heat & Power Plant,3629,22199,U S Air Force-Eielson AFB,3547,2,coal,NaN,NaN,32.129356,NaN
72839,1391,2011-12-01,2A,Louisiana 1,360,55936,Entergy Gulf States Louisiana LLC,4208,1,gas,False,NaN,13.187648,NaN
85302,6504,2011-07-01,15,Maalaea,1177,11843,Maui Electric Co Ltd,176,1,oil,False,24.57953,NaN,NaN
250124,50002,2015-09-01,GEN2,Pittsfield Generating LP,3453,15114,"Pittsfield Generating Company, LP",2859,1,gas,NaN,NaN,8.567076,NaN
4450,898,2009-11-01,5,Wood River,1623,5517,Dynegy Midwest Generation Inc,1554,2,coal,False,NaN,10.118172,NaN


### Per-generator Capacity Factor

In [44]:
%%time
pudl_out_monthly.capacity_factor().sample(n_samples)

CPU times: user 771 ms, sys: 3.54 ms, total: 775 ms
Wall time: 805 ms


,plant_id_eia,report_date,generator_id,net_generation_mwh,capacity_mw,capacity_factor
116110,715,2012-05-01,1,-145.00,50.0,-0.003898
283858,55470,2015-12-01,ST805,16114.00,250.0,0.086634
249099,50087,2015-04-01,GEN3,1411.00,5.0,0.391944
451237,54409,2019-06-01,GEN2,328.83,2.0,0.228354
411479,50776,2018-08-01,GEN1,0.00,94.0,0.000000


### Per-generator MCOE
* This function uses the cached dataframes that were generated above to produce a huge table of per-generator statistics.
* If you just called this function alone, all of those other dataframes would be automatically generated, and available within the output object.

In [45]:
%%time
pudl_out_monthly.mcoe().sample(n_samples)

CPU times: user 23.6 s, sys: 7.51 s, total: 31.1 s
Wall time: 31.4 s


,report_date,plant_id_eia,plant_id_pudl,unit_id_pudl,generator_id,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,associated_combined_heat_power,...,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
296279,2016-12-01,50006,3455,1.0,GTG5,Linden Cogen Plant,3890,1597,EFS Cogen Holdings I LLC,True,...,5.613890e+05,None,NaN,None,None,False,None,100.0,NaN,07036
361905,2017-10-01,59338,7157,2.0,CTG1,PB1 - Gila River,60459,4621,CXA Sundevil Power I,False,...,NaN,None,NaN,None,None,False,None,163.0,NaN,85337
118803,2012-04-01,6052,661,2.0,2,Wansley,7140,123,Georgia Power Co,False,...,1.632797e+06,None,NaN,None,None,None,None,872.0,NaN,30170
154192,2013-05-01,6113,221,4.0,4,Gibson,15470,92,PSI Energy Inc,False,...,3.126066e+06,None,NaN,None,None,False,None,627.0,NaN,47665
455720,2013-02-01,54618,<NA>,NaN,900,NaN,<NA>,<NA>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Free Memory

In [46]:
del pudl_out_monthly

# Preliminary Output Tables 
* Integrating a new dataset into the PUDL database requires many steps (datastore, extract, transform, load, outputs).
* Sometimes we need to use tables from new datasets as soon as possible for analysis.
* The interim extract and transform steps can be hacked into the output object to run on the fly, prior to DB integration.
* The data extraction and transformation can take a while though -- and it will need to be re-run from scratch every time you create a new output object.
* **WARNING:** None of this data has been fully validated, and the structure is likely to change. Some of it (especially the FERC 714) is still in a pretty raw state.

As of December 2020, we have preliminarily integrated EIA 861 and FERC 714 in this format.

## EIA Form 861
* The interim EIA 861 ETL is set up to automatically run in its entirety as soon as you request any EIA 861 table.
* This should take 2-5 minutes if you already have the raw input data avaialble.
* If raw input data needs to be downloaded [from our Zenodo archives](https://zenodo.org/record/4127029) first (which should happen automatically), it will take longer.

In [47]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [48]:
# here are all of the EIA 861 tables
methods_eia861 = [t for t in methods_pudl_out if '_eia861' in t and "etl" not in t]
methods_eia861

['advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'demand_side_management_eia861',
 'distributed_generation_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_eia861',
 'non_net_metering_eia861',
 'operational_data_eia861',
 'reliability_eia861',
 'sales_eia861',
 'service_territory_eia861',
 'utility_assn_eia861',
 'utility_data_eia861']

### EIA 861 Balancing Authorities

In [49]:
%%time
pudl_out.balancing_authority_eia861().sample(n_samples)

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:35: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo

,report_date,balancing_authority_id_eia,balancing_authority_code_eia,balancing_authority_name_eia
12431,2005-01-01,10171,NaN,LG&E Energy Transmission
9192,2004-01-01,12506,NaN,Entergy Electric System
10304,2004-01-01,599,AMPL,"Anchorage, Municipality of"
18178,2006-01-01,19281,TIDC,Turlock Irrigation District
15568,2006-01-01,599,AMPL,"Anchorage, Municipality of"


### EIA 861 Advanced Metering Infrastructure

In [50]:
%%time
pudl_out.advanced_metering_infrastructure_eia861().sample(n_samples)

CPU times: user 4.91 ms, sys: 26 µs, total: 4.93 ms
Wall time: 4.34 ms


,utility_id_eia,state,balancing_authority_code_eia,report_date,entity_type,short_form,utility_name_eia,customer_class,advanced_metering_infrastructure,automated_meter_reading,daily_digital_access_customers,direct_load_control_customers,energy_served_ami_mwh,home_area_network,non_amr_ami
68815,4117,VA,PJM,2016-01-01,NaN,<NA>,Community Electric Coop,commercial,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,1800
59066,8034,OH,PJM,2015-01-01,NaN,<NA>,Hancock-Wood Electric Coop Inc,industrial,13,72,<NA>,<NA>,NaN,<NA>,<NA>
115927,20729,MD,PJM,2019-01-01,Municipal,<NA>,Town of Williamsport - (MD),residential,<NA>,851,<NA>,<NA>,NaN,<NA>,<NA>
66352,27599,IN,MISO,2015-01-01,NaN,<NA>,Carroll-White REMC,residential,14349,<NA>,14358,2881,156307.0,<NA>,9
97727,14563,MS,MISO,2018-01-01,NaN,<NA>,Pearl River Valley El Pwr Assn,residential,48031,<NA>,48047,0,734865.0,<NA>,16


### EIA 861 Sales
How much electricity did utilities report selling to different types of customers in each year by state?

In [51]:
%%time
pudl_out.sales_eia861().sample(n_samples)

CPU times: user 17.5 ms, sys: 44 µs, total: 17.6 ms
Wall time: 16.9 ms


,utility_id_eia,state,report_date,balancing_authority_code_eia,business_model,data_observed,entity_type,service_type,short_form,utility_name_eia,customer_class,customers,sales_mwh,sales_revenue
398532,56734,TX,2019-01-01,ERCO,energy_services,True,Retail Power Marketer,bundled,<NA>,Smart Prepaid Electric,commercial,0.0,0.0,0.0
315650,59815,PA,2015-01-01,PJM,retail,True,Retail Power Marketer,energy,<NA>,"HIKO Energy, LLC",other,NaN,NaN,NaN
81072,16655,CA,2004-01-01,UNK,retail,True,Municipal,bundled,<NA>,City of Santa Clara,commercial,5941.0,108589.0,11286000.0
67640,3082,MO,2004-01-01,UNK,retail,True,Municipal,bundled,<NA>,Carrollton Board of Public Wks,other,NaN,NaN,NaN
40335,19716,KY,2002-01-01,UNK,retail,True,Municipal,bundled,<NA>,Vanceburg City of,residential,2404.0,31379.0,1788000.0


### EIA 861 Service Territories
Which counties (with FIPS codes) each utility reported serving in each year.

In [52]:
%%time
pudl_out.service_territory_eia861().sample(n_samples)

CPU times: user 12.1 ms, sys: 114 µs, total: 12.2 ms
Wall time: 11.6 ms


,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips
37607,Noble,<NA>,OH,18085,South Central Power Company,2004-01-01,39,39121
20661,Turner,<NA>,SD,19293,Southeastern Electric Coop Inc,2002-01-01,46,46125
6776,Victoria,<NA>,TX,5139,"Dewitt Electric Coop, Inc",2001-01-01,48,48469
78144,Burnet,<NA>,TX,14626,"Pedernales Electric Coop, Inc",2007-01-01,48,48053
170901,Garden,<NA>,NE,13337,Nebraska Public Power District,2015-01-01,31,31069


### Free Memory

In [53]:
del pudl_out

## FERC Form 714
* **NOTE:** Most of the FERC Form 714 tables have not yet been fully processed.
* We have primarily been focused on the historical hourly demand reported by planning areas.
* As with the EIA 861, the full interim ETL will be run as soon as you ask for any FERC 714 table.
* Also as with the EIA 861, if you don't have the [raw FERC 714 input files](https://zenodo.org/record/4127101) cached locally already, they might take a minute to download.

In [54]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [55]:
# here are all of the FERC 714 tables
methods_ferc714 = [t for t in methods_pudl_out if '_ferc714' in t and "etl" not in t]
methods_ferc714

['adjacency_ba_ferc714',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'description_pa_ferc714',
 'gen_plants_ba_ferc714',
 'id_certification_ferc714',
 'interchange_ba_ferc714',
 'lambda_description_ferc714',
 'lambda_hourly_ba_ferc714',
 'net_energy_load_ba_ferc714',
 'respondent_id_ferc714']

### FERC 714 Respondents
Currently the processing of the hourly planning area demand table exceeds the available memory on this JupyterHub, so the following cells are commented out.

In [56]:
%%time
respondent_id_ferc714 = pudl_out.respondent_id_ferc714()
respondent_id_ferc714.sample(5)

Running the interim FERC 714 ETL process!
Extracting respondent_id_ferc714 from CSV into pandas DataFrame.
Extracting id_certification_ferc714 from CSV into pandas DataFrame.
Extracting gen_plants_ba_ferc714 from CSV into pandas DataFrame.


/home/zane/code/catalyst/pudl/src/pudl/extract/ferc714.py:59: UserWarning: Integration of FERC 714 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


Extracting demand_monthly_ba_ferc714 from CSV into pandas DataFrame.
Extracting net_energy_load_ba_ferc714 from CSV into pandas DataFrame.
Extracting adjacency_ba_ferc714 from CSV into pandas DataFrame.
Extracting interchange_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_hourly_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_description_ferc714 from CSV into pandas DataFrame.
Extracting description_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_forecast_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_hourly_pa_ferc714 from CSV into pandas DataFrame.
Transforming respondent_id_ferc714.
Transforming id_certification_ferc714.
Transforming gen_plants_ba_ferc714.
Transforming demand_monthly_ba_ferc714.
Transforming net_energy_load_ba_ferc714.
Transforming adjacency_ba_ferc714.
Transforming interchange_ba_ferc714.
Transforming lambda_hourly_ba_ferc714.
Transforming lambda_description_ferc714.
Transforming description_pa_ferc714.
Transfor

,respondent_id_ferc714,respondent_name_ferc714,eia_code
109,218,Oglethorpe Power Company,13994
150,259,Southwestern Public Service Company (Xcel),17718
162,272,Western Area Power Admin - Upper Missouri-East...,28502
114,223,Omaha Public Power District,14127
187,298,Gulf Power Company,7801


### FERC 714 Hourly Demand by Planning Area

In [57]:
demand_hourly_pa_ferc714 = pudl_out.demand_hourly_pa_ferc714()
demand_hourly_pa_ferc714.sample(20)

,respondent_id_ferc714,report_date,utc_datetime,timezone,demand_mwh
1672275,238,2014-01-01,2014-10-14 10:00:00,America/Los_Angeles,112.00
14512642,219,2010-01-01,2010-09-30 04:00:00,America/New_York,48.00
1048386,102,2014-01-01,2014-07-10 07:00:00,America/Chicago,6619.00
14698040,234,2014-01-01,2014-05-10 04:00:00,America/New_York,4434.00
2828427,263,2012-01-01,2012-05-16 10:00:00,America/Chicago,14576.00
12011001,238,2010-01-01,2010-01-09 03:00:00,America/Los_Angeles,229.00
13859316,262,2009-01-01,2009-11-03 03:00:00,America/New_York,2043.00
1396265,149,2008-01-01,2008-02-03 08:00:00,America/Chicago,510.00
13710166,219,2007-01-01,2007-08-15 03:00:00,America/New_York,44.00
11119831,157,2018-01-01,2018-11-24 22:00:00,America/New_York,12059.00


# Future Analyses
The output object contains a lot of different kinds of things, and as we accumulate more and different kinds of analyses in our library, we're looking to break them out into their own reusable classes that access the database directly. Some work in progress here is related to constructing historical service territory geometries for both utilities and balancing authorities, and associating that data usefully with the FERC 714 respondents. Exploring that in detail is beyond the scope of this notebook, but check out the `pudl.output.ferc714` and `pudl.analysis.service_territory` modules for examples. Unfortunately as this analysis currently depends on both the interim EIA 861 and the interim FERC 714 datasets, it uses too much memory to be run on the JupyterHub right now.

In [58]:
%%time
ferc714_out = pudl.output.ferc714.Respondents(pudl_out)
annualized = ferc714_out.annualize()
categorized = ferc714_out.categorize()
summarized = ferc714_out.summarize_demand()
fipsified = ferc714_out.fipsify()
counties_gdf = ferc714_out.georef_counties()

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:35: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo

/home/zane/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


CPU times: user 3min 27s, sys: 4.3 s, total: 3min 31s
Wall time: 3min 40s
